In [190]:
#Import data
import pandas as pd
from datetime import datetime
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import circmean

alpha = 0.05

#path of a boat_log
csv_path = r"C:\Users\matsa\OneDrive\Υπολογιστής\SailGP\SGP Data challenge VS\SGP_Data_Challenge\Data\2025-01-19_man_summary.csv"

df = pd.read_csv(csv_path)
df["TIME_LOCAL_unk"] = pd.to_datetime(df["TIME_LOCAL_unk"])
#df=df.sort_values(by='DATETIME', ascending= True)
df = df.set_index('TIME_LOCAL_unk')

In [191]:
df['twa_delta']=df.apply(lambda row: circmean([circular_diff(row['entry_twa'], row['exit_twa'])]), axis=1)
df['orig_twa_delta']=df.apply(lambda row: circmean([circular_diff(row['orig_entry_twa'], row['orig_exit_twa'])]), axis=1)
df['delta_bsp']=df.apply(lambda row: row['entry_bsp'] - row['exit_bsp'], axis=1)

# Bins TWS

In [192]:
bin_width = 4
min_tws = int(df['tws'].min())
max_tws = int(df['tws'].max())+1

# Create bins with range from min to max, every 2 kmh
df['tws_bins']=pd.cut(df['tws'], bins=range(min_tws, max_tws + bin_width, bin_width))


In [193]:
def circular_diff(angle1, angle2):
    # Normalize the difference to the range [-180, 180]
    diff = (angle1 - angle2 + 180) % 360 - 180
    return diff


In [194]:
df['isvalid'] = (
    (df['delta_bsp'] < 5) & 
    (df['twa_delta'].abs() < 5) & 
    (df['flying'] == 1)
)


In [195]:
df_valid=df[df['isvalid']==True]

In [196]:
df_valid['flying'].max()

1

In [203]:
len(df_valid)

74

In [204]:
columns_to_average= df_valid.select_dtypes(include='number').columns
averages = df_valid.groupby(['BOAT','type','tws_bins'])[columns_to_average].mean().round(1)
averages = averages.sort_values(by='BOAT', ascending=True)


C:\Users\matsa\AppData\Local\Temp\ipykernel_19408\1960434239.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  averages = df_valid.groupby(['BOAT','type','tws_bins'])[columns_to_average].mean().round(1)


In [205]:
columns_to_exclude = ['WING_CONFIG_unk', 'MD4_SEL_DB_unk', 'MD4_SEL_RUD_unk', 'race', 'leg','htw_bsp']

# Drop the unwanted columns before grouping
averages = averages.drop(columns=columns_to_exclude)

In [206]:
averages=averages[['entry_bsp', 'exit_bsp', 'min_bsp', 'bsp_loss', 'entry_twa', 'exit_twa','orig_entry_twa', 'orig_exit_twa','twa_delta',
       'orig_twa_delta', 'flying']].dropna()

In [207]:
averages

entry_bsp  exit_bsp  min_bsp  bsp_loss  entry_twa  \
BOAT type tws_bins                                                      
AUS  gybe (31, 35]       79.3      74.4     50.6      28.6      149.7   
          (35, 39]       75.5      74.4     55.9      19.6      151.3   
          (39, 43]       74.0      80.4     55.8      18.2      144.7   
     tack (31, 35]       51.4      54.8     39.3      12.1       38.7   
          (35, 39]       58.9      55.7     39.9      19.1       43.1   
          (39, 43]       55.7      56.0     39.5      16.2       40.0   
BRA  tack (31, 35]       49.3      52.1     37.8      11.5       45.0   
     gybe (35, 39]       75.0      73.3     53.2      21.8      149.4   
          (31, 35]       74.0      71.2     54.5      19.5      150.8   
DEN  tack (35, 39]       55.1      52.5     39.4      15.8       49.4   
     gybe (35, 39]       73.7      69.9     62.8      11.0      141.6   
ESP  gybe (31, 35]       76.5      77.6     54.4      22.2      154.0   
          (35, 39]       75.4      73.4     56.3      19.1      151.8   
          (39, 43]       77.7      75.7     57.1      20.7      151.1   
     tack (31, 35]       59.0      58.2     41.8      17.2       42.9   
          (35, 39]       46.1      53.3     37.3       8.8       41.3   
          (39, 43]       58.0      57.1     39.4      18.7       34.7   
GBR  tack (35, 39]       56.8      53.4     40.3      16.5       41.9   
          (39, 43]       52.2      61.0     39.3      12.9       36.6   
          (31, 35]       59.3      59.1     39.9      19.4       42.3   
     gybe (39, 43]       77.8      80.2     57.1      20.7      141.6   
          (35, 39]       78.0      78.9     54.2      23.8      150.4   
          (31, 35]       72.4      73.8     53.9      18.6      143.1   
GER  tack (31, 35]       53.9      69.6     42.4      11.6       79.1   
          (39, 43]       48.9      53.7     37.6      11.3       51.0   
ITA  tack (35, 39]       38.3      55.3     33.3       5.0       51.9   
          (31, 35]       55.2      61.4     37.2      18.0       48.0   
          (39, 43]       48.6      51.7     35.2      13.4       40.4   
     gybe (39, 43]       80.9      79.4     56.7      24.2      155.2   
          (35, 39]       77.5      79.1     56.8      20.7      152.2   
          (31, 35]       70.4      72.4     52.5      17.9      121.8   
NZL  tack (39, 43]       53.8      50.7     39.7      14.1       41.8   
          (31, 35]       51.5      49.3     37.5      14.0       44.6   
          (35, 39]       57.0      60.3     42.9      14.1       47.3   
     gybe (39, 43]       77.4      76.7     57.6      19.7      145.7   
          (35, 39]       74.8      76.2     54.4      20.4      156.3   
SUI  gybe (31, 35]       70.1      73.5     55.0      15.0      147.2   
          (35, 39]       73.9      75.1     52.9      21.0      143.8   
          (39, 43]       76.9      74.1     52.9      24.0      140.3   
     tack (31, 35]       52.6      49.6     35.8      16.8       57.3   
          (35, 39]       37.7      48.7     34.2       3.5       46.7   

                    exit_twa  orig_entry_twa  orig_exit_twa  twa_delta  \
BOAT type tws_bins                                                       
AUS  gybe (31, 35]     145.7           149.7         -145.7        4.0   
          (35, 39]     149.8            92.1          -88.8        2.8   
          (39, 43]     155.2           144.7         -155.2        2.1   
     tack (31, 35]      43.2           -38.7           43.2        1.8   
          (35, 39]      48.3            -0.1           -5.4        1.0   
          (39, 43]      47.3            40.0          -47.3        3.2   
BRA  tack (31, 35]      40.8           -45.0           40.8        4.2   
     gybe (35, 39]     146.8           149.4         -146.8        2.6   
          (31, 35]     144.1           150.8         -144.1        0.4   
DEN  tack (35, 39]      47.1             1.7          -16.3       

In [209]:
averages.to_excel("Calibration_averages from man.xlsx")